In [1]:
# Sources: https://github.com/jiaeyan/Jiayan
#          https://github.com/kpu/kenlm

In [2]:
import pandas as pd
from jiayan import load_lm
from jiayan import CharHMMTokenizer
from jiayan import CRFPOSTagger
from jiayan import CRFSentencizer
import numpy as np

## An Example of Jiayan
### Jiayan POS Tags

Tag | Description | Example
--- | --- | ---
a | adjective | 幽明
b | other noun-modifier | 男，女
c | conjunction | 与，而
d | adverb | 皆
e | exclamation | 呜呼
g | morpheme | 甥
h | prefix | 非
i | idiom | 发愤忘食
j | abbreviation | 五帝
k | suffix | 者
m | number | 一，百
n | general noun | 鬼神，山川
nd | direction noun | 东，西，南，北
nh | person name | 轩辕
ni | organization name | 辽队
nl | location noun | 城北
ns | geographical name | 襄平县
nt | temporal noun | 春，夏，秋，冬
nz | other proper noun | 山海经
o | onomatopoeia | 呜呜
p | preposition | 以，为
q | quantity | 年，岁
r | pronoun | 其，斯
u | auxiliary | 之，所
v | verb | 赐
wp | punctuation | ，。！
ws | foreign words | CPU
x | non-lexeme | 萄, 翱
z | descriptive words | 默然，区区

In [3]:
text1 = '别来春半，触目柔肠断。砌下落梅如雪乱，拂了一身还满。雁来音信无凭，路遥归梦难成。离恨恰如春草，更行更远还生。'
text2 = '别来春半触目柔肠断砌下落梅如雪乱拂了一身还满雁来音信无凭路遥归梦难成离恨恰如春草更行更远还生'

lm = load_lm('/Users/jojoli/jiayan_models/jiayan.klm')
tokenizer = CharHMMTokenizer(lm)
words = list(tokenizer.tokenize(text1))
print(words)

postagger = CRFPOSTagger()
postagger.load('/Users/jojoli/jiayan_models/pos_model')
print('\n', postagger.postag(words))

sentencizer = CRFSentencizer(lm)
sentencizer.load('/Users/jojoli/jiayan_models/cut_model')
print('\n', sentencizer.sentencize(text2))

postagger.postag("抽")

['别来', '春', '半', '，', '触目', '柔', '肠断', '。', '砌下', '落梅', '如雪', '乱', '，', '拂', '了一', '身', '还', '满', '。', '雁来', '音信', '无', '凭', '，', '路遥', '归', '梦难成', '。', '离恨', '恰如', '春草', '，', '更', '行', '更远', '还', '生', '。']

 ['nh', 'nt', 'm', 'wp', 'v', 'a', 'n', 'wp', 'v', 'v', 'n', 'a', 'wp', 'v', 'v', 'n', 'd', 'a', 'wp', 'v', 'n', 'v', 'n', 'wp', 'n', 'v', 'n', 'wp', 'v', 'v', 'n', 'wp', 'd', 'v', 'n', 'd', 'v', 'wp']

 ['别来春半', '触目柔肠断砌下落', '梅如雪乱', '拂了一身还满', '雁来音信', '无凭路遥', '归梦难成离恨', '恰如春草', '更行更远还生']


['v']

## Preprocessing

In [4]:
columns = ["type", "content"]
data = pd.read_csv("/Users/jojoli/Documents/夏校申请:项目制作/英才计划/正式培养/chinese/datasets/Tang_Dynasty.txt",
                    delimiter=":", header=None,names=columns,skiprows=0)
data.head()

,type,content
0,首春,寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。
1,初晴落景,晚霞聊自怡，初晴弥可喜。日晃百花色，风动千林翠。池鱼跃不同，园鸟声还异。寄言博通者，知予物外志。
2,初夏,一朝春夏改，隔夜鸟花迁。阴阳深浅叶，晓夕重轻烟。哢莺犹响殿，横丝正网天。珮高兰影接，绶细草纹...
3,度秋,夏律昨留灰，秋箭今移晷。峨嵋岫初出，洞庭波渐起。桂白发幽岩，菊黄开灞涘。运流方可叹，含毫属微理。
4,仪鸾殿早秋,寒惊蓟门叶，秋发小山枝。松阴背日转，竹影避风移。提壶菊花岸，高兴芙蓉池。欲知凉气早，巢空燕不窥。


In [5]:
# Remove Punctuations

punct_list = list("。" + "，" + "；" + "（" + "）" + "《" + "》")
def remove_punctuation(text):
    for punc in punct_list:
        if punc in text:
            text = text.replace(punc, " ")
    return text.strip()

data['clean_content'] = data["content"].astype(str).apply(remove_punctuation)

In [6]:
# Remove stopwords

def re_stop(text):
    for chr in text:
        ls = postagger.postag(chr)
        if ls[0] == 'p' or ls[0] == 'r' or ls[0] == 'u' or ls[0] == 'c':
            text = text.replace(chr, "")
    return text.strip()

data['nostop_clean_content'] = data["clean_content"].apply(re_stop)

In [7]:
data.iloc[13:14, 3:4]

,nostop_clean_content
13,昔年怀壮气 提戈初仗节 心随朗日高 志秋霜洁 移锋惊电起 转战长河决 营碎落星沉 阵卷横云裂...


In [8]:
data.tail()

,type,content,clean_content,nostop_clean_content
43025,促拍满路花,西风吹渭水，落叶满长安。茫茫尘世里，独清闲。自然炉鼎，虎绕与龙盘。九转丹砂就，一粒刀圭，便成...,西风吹渭水 落叶满长安 茫茫尘世里 独清闲 自然炉鼎 虎绕与龙盘 九转丹砂就 一粒刀圭 便成...,西风吹渭水 落叶满长安 茫茫尘世里 独清闲 自炉鼎 虎绕龙盘 九转丹砂就 一粒刀圭 便成陆地...
43026,六么令,东与西，眼与眉。偃月炉中运坎离，灵砂且上飞。最幽微，是天机，你休痴，你不知。,东与西 眼与眉 偃月炉中运坎离 灵砂且上飞 最幽微 是天机 你休痴 你不知,东西 眼眉 偃月炉中运坎离 灵砂上飞 最幽微 是天机 休痴 不知
43027,汉宫春,横笛声沉，倚危楼红日，江转天斜。黄尘边火澒洞，何处吾家。胎禽怨夜，来乘风、玄露丹霞。先生笑、...,横笛声沉 倚危楼红日 江转天斜 黄尘边火澒洞 何处吾家 胎禽怨夜 来乘风、玄露丹霞 先生笑、...,横笛声沉 倚危楼红日 江转天斜 黄尘边火澒洞 何处家 胎禽怨夜 来乘风、玄露丹霞 先生笑、飞...
43028,忆江南,江南鼓，梭肚两头栾。钉著不知侵骨髓，打来只是没心肝。空腹被人谩。,江南鼓 梭肚两头栾 钉著不知侵骨髓 打来只是没心肝 空腹被人谩,江南鼓 梭肚两头栾 钉著不知侵骨髓 打来只是没心肝 空腹人谩
43029,句,暂游大庾，白鹤飞来谁共语？岭畔人家，曾见寒梅几度花。春来春去，人在落花流水处。花满前蹊，藏尽...,暂游大庾 白鹤飞来谁共语？岭畔人家 曾见寒梅几度花 春来春去 人在落花流水处 花满前蹊 藏尽...,暂游大庾 白鹤飞来共语？岭畔人家 曾见寒梅几度花 春来春去 人落花流水处 花满前蹊 藏尽神仙人不知


### POS tagging & Noun Extraction

In [10]:
def extract_noun(text):
    text = list(tokenizer.tokenize(text))
    temp = []
    for each in text:
        ls = postagger.postag(each)
        if (ls[0] == 'n'):
            temp.append(each) # extract nouns
    return temp

data['all_nouns'] = data["nostop_clean_content"].apply(extract_noun)
data.head()

,type,content,clean_content,nostop_clean_content,all_nouns
0,首春,寒随穷律变，春逐鸟声开。初风飘带柳，晚雪间花梅。碧林青旧竹，绿沼翠新苔。芝田初雁去，绮树巧莺来。,寒随穷律变 春逐鸟声开 初风飘带柳 晚雪间花梅 碧林青旧竹 绿沼翠新苔 芝田初雁去 绮树巧莺来,寒随穷律变 春逐鸟声开 初风飘带柳 晚雪间花梅 碧林青旧竹 绿沼翠新苔 芝田初雁去 绮树巧莺来,"[律, 鸟声, 风飘, 柳, 雪, 碧林, 竹, 沼翠新苔, 雁去]"
1,初晴落景,晚霞聊自怡，初晴弥可喜。日晃百花色，风动千林翠。池鱼跃不同，园鸟声还异。寄言博通者，知予物外志。,晚霞聊自怡 初晴弥可喜 日晃百花色 风动千林翠 池鱼跃不同 园鸟声还异 寄言博通者 知予物外志,晚霞聊自怡 初晴弥可喜 日晃百花色 风动千林翠 池鱼跃不 园鸟声还异 寄言博通者 知予物外志,"[色, 风动, 池, 鱼跃, 园, 鸟声, 言, 物外, 志]"
2,初夏,一朝春夏改，隔夜鸟花迁。阴阳深浅叶，晓夕重轻烟。哢莺犹响殿，横丝正网天。珮高兰影接，绶细草纹...,一朝春夏改 隔夜鸟花迁 阴阳深浅叶 晓夕重轻烟 哢莺犹响殿 横丝正网天 珮高兰影接 绶细草纹...,一朝春夏改 隔夜鸟花迁 阴阳深浅叶 晓夕重轻烟 哢莺犹响殿 横丝正网天 珮高兰影接 绶细草纹...,"[鸟, 殿, 丝, 网, 天, 珮高兰影, 纹, 碧, 鳞惊棹侧, 檐前, 处, 山, 泉]"
3,度秋,夏律昨留灰，秋箭今移晷。峨嵋岫初出，洞庭波渐起。桂白发幽岩，菊黄开灞涘。运流方可叹，含毫属微理。,夏律昨留灰 秋箭今移晷 峨嵋岫初出 洞庭波渐起 桂白发幽岩 菊黄开灞涘 运流方可叹 含毫属微理,夏律昨留灰 秋箭今移晷 峨嵋岫初出 洞庭波渐起 桂白发幽岩 菊黄开灞涘 运流方可叹 含毫属微理,"[律, 箭, 洞庭, 波, 菊黄, 理]"
4,仪鸾殿早秋,寒惊蓟门叶，秋发小山枝。松阴背日转，竹影避风移。提壶菊花岸，高兴芙蓉池。欲知凉气早，巢空燕不窥。,寒惊蓟门叶 秋发小山枝 松阴背日转 竹影避风移 提壶菊花岸 高兴芙蓉池 欲知凉气早 巢空燕不窥,寒惊蓟门叶 秋发小山枝 松阴背日转 竹影避风移 提壶菊花岸 高兴芙蓉池 欲知凉气早 巢空燕不窥,"[枝, 松阴, 竹影, 菊花, 岸, 池, 巢空]"


### Randomly Select 600 Poems to be Saved and Tagged

In [11]:
(data.sample(n=600)).to_csv("/Users/jojoli/Documents/夏校申请:项目制作/英才计划/正式培养/chinese/datasets/Tang_Dynasty_preprocessed.csv", index=False)